In [7]:
import sys 
sys.path.append("../")

import torch 
import utils 
import nerf_model

In [12]:
import itertools 
import numpy as np

im_w = 5
im_h = 10 
idxs = [(i,j) for i,j in itertools.product(np.arange(0,im_h), np.arange(0,im_w))]
idxs = torch.FloatTensor(idxs)
print(idxs)
# print(idxs.shape)


tensor([[0., 0.],
        [0., 1.],
        [0., 2.],
        [0., 3.],
        [0., 4.],
        [1., 0.],
        [1., 1.],
        [1., 2.],
        [1., 3.],
        [1., 4.],
        [2., 0.],
        [2., 1.],
        [2., 2.],
        [2., 3.],
        [2., 4.],
        [3., 0.],
        [3., 1.],
        [3., 2.],
        [3., 3.],
        [3., 4.],
        [4., 0.],
        [4., 1.],
        [4., 2.],
        [4., 3.],
        [4., 4.],
        [5., 0.],
        [5., 1.],
        [5., 2.],
        [5., 3.],
        [5., 4.],
        [6., 0.],
        [6., 1.],
        [6., 2.],
        [6., 3.],
        [6., 4.],
        [7., 0.],
        [7., 1.],
        [7., 2.],
        [7., 3.],
        [7., 4.],
        [8., 0.],
        [8., 1.],
        [8., 2.],
        [8., 3.],
        [8., 4.],
        [9., 0.],
        [9., 1.],
        [9., 2.],
        [9., 3.],
        [9., 4.]])


In [16]:
def photo_nerf_to_image(model, im_h, im_w): 
    idxs = [(i,j) for i,j in itertools.product(np.arange(0,im_h), np.arange(0,im_w))]
    idxs = torch.FloatTensor(idxs)
    N, _ = idxs.shape
    recon = []
    step = 4096
    for i in range(0, N, step):
        # break up whole tensor into sizeable chunks
        batch = idxs[i:i+step,:]
        rgb = model(batch)
        recon.append(rgb)
    recon = torch.cat(recon, axis=0).reshape((im_h, im_w, 3))
    return recon

model = nerf_model.ImageNeRFModel()
recon = photo_nerf_to_image(model, 400, 500)
print(recon.shape)

torch.Size([400, 500, 3])
